## Modelado de opiniones y su impacto en la generación de consensos y disensos en una plataforma de CrowdSourcing

El objetivo de la simulación es dilucidar el funcionamiento de las dinámicas de opinión en una plataforma de CrowdSourcing donde las personas expresan sus opiniones y votan las expresadas por otros bajo una serie de condiciones especificadas por el contexto de la plataforma.

In [1]:
import random, numpy as np, pandas as pd, matplotlib.pyplot as plt, math, scipy
from scipy.stats import multivariate_normal
from scipy.spatial import distance
from pandas import Series, DataFrame
from uuid import uuid4
#pd.options.mode.chained_assignment = None 

TamañoDeLaMuestra = [100, 200, 500, 1000] # Cantidad de agentes que quiero en la muestra.

# Agentes

## Agente

Agente es una clase que cuenta con los siguientes atributos:

* **Oi** = Contiene la opinion de cada agente. Va a ser siempre numerico pero dependiendo la simulacion puede ser un solo numero (real) o un vector de varios numeros (reales), independientemente de como se generan que siempre lo hacen a partir de la realizacion de algun tipo de distribucion conocida en un espacio univariado o multivariado.  
* **ID** = Es un numero entero de identificacion unico para cada sujeto.
* **IdeasVistas** = Es una lista que contiene los IDs de las ideas que va a ver este agente.  
* **IdeasVotadas** = Es una lista que contiene los IDs de las ideas que va a ver este agente.
* **Valencias** = Es una lista que contiene el valor que toma el voto de este agente.

In [2]:
class Agente (object):
    def __init__ (self, tipo, media, ID, de = 0.5, dimensiones = 3):
        self.Oi = self.set_Oi (tipo, media, de, dimensiones)
        self.ID = ID
        self.IdeasVistas = []
        self.IdeasVotadas = []
        self.Valencias = []
        self.Tipo = tipo
        self.Media = media
        self.DE = de

    def set_Oi (self, tipo, media, de, dimensiones): 
        if tipo not in ['uniforme', 'univariada', 'multivariada']:
            raise Exception("El tipo de opinion solo puede ser ""uniforme"", ""univariada"" o ""multivariada")
        
        elif isinstance(media, list) == False:
            raise Exception("El argumento media solo acepta listas")
        
        opinion = []
        mediasu = []
        
        if tipo == "uniforme":
            if len(media) != 2:
                raise Exception("La media del tipo uniforme debe contener un start y un end")
            opinion.append (random.uniform (media[0],media[1]))
            mediasu += media 
            
        if tipo == "univariada":
            if len(media) != 1:
                raise Exception("La media de la univariada no puede contener mas de un numero")
            opinion = np.random.normal (media,de)
            mediasu = media
                    
        if tipo == "multivariada":
            if len(media) == 1:
                for i in range (dimensiones):
                     mediasu += media
                a = multivariate_normal.rvs(mediasu, de)
                opinion = a.tolist()
                    
            else:
                a = multivariate_normal.rvs(media, de)
                opinion = a.tolist()
   
        return tipo, mediasu, de, opinion

    def append_IdeasVistas (self, IdeaVista):
        self.IdeasVistas.append (IdeaVista)

    def append_IdeasVotadas (self, IdeaVotada):
        self.IdeasVotadas.append (IdeaVotada)   

    def append_Valencias (self, Valencias):
        self.Valencias.append (Valencias) 

    def ListaVotos (self, VotosPos, VotosNeg):
        votos = VotosPos * [1] + VotosNeg * [-1]
        return votos

    def comparar (self, idea): # Devuelve la distancia entre la opinion que se le pase y la del sujeto
        Distancia = abs(idea-self.Oi)
        return Distancia
    
    def get_Oi (self):
        return self.Oi
    
    def get_IdeasVistas (self):
        return self.IdeasVistas

    def get_ID (self):
        return self.ID
    
    def clean_IdeasVistas (self):
        self.IdeasVistas = []

### Agente Nulo

In [3]:
class AgenteNulo (Agente):
    
    def votar (self, ideas): # Modelo de votacion aleatoria nulo
        self.IdeasVistas.append (ideas)
        Votara = ideas.sample (n = QIdeasAVotar)
        votos = self.ListaVotos(VotosPos, VotosNeg)
        vota = []
        for i in Votara.index:
            vota.append (i)
        self.IdeasVotadas.append (vota)
        self.Valencias.append (votos)
        devuelve = pd.DataFrame(votos, vota)
        return devuelve

## Agente Determinista

In [4]:
class AgenteDeterminista (AgenteNulo):

    def algvotar (self, ideas, tipo): # Asume que se le da un dataframe con las opiniones (distribuidas uniformemente),
        Comp = 3             # y devuelve cual es el ID de la idea mas cercana a la del sujeto
        vota = 0
        for i in ideas.index:
            V = abs(ideas.at[i,'Opinion']-self.Oi)
            if V == Comp:
                vota = random.randint (i, vota)
            elif tipo == "Positivo":
                if V < Comp:
                    vota = i
                    Comp = V
            elif tipo == "Negativo":
                if V > Comp:
                    vota = i
                    Comp = V                
        return vota # Me devuelve la id de la idea a votar
        
        def votar (self, ideas):
            self.IdeasVistas.append (ideas)
            ideas2 = ideas.copy ()
            vota = []
            votos = self.ListaVotos(VotosPos, VotosNeg)
            for i in votos:
                if i == 1:
                    va = self.algvotar (ideas2, "Positivo")
                    ideas2.drop (va)
                    vota.append (va)
                if i == -1:
                    va = self.algvotar (ideas2, "Negativo")
                    ideas2.drop (va)
                    vota.append (va)
            self.IdeasVotadas.append (vota)
            self.Valencias.append (votos)
            devuelve = pd.DataFrame(votos, vota)
            return devuelve

## Agente Probabilistico

In [5]:
class AgenteProbabilistico (AgenteNulo): ####<<<<<<<<<<<<<<<<< Estoy trabajandola aun

    def algvotar (self, ideas, tipo):
        Comp = 3
        vota = 0
        for i in ideas.index:
            V = abs(ideas.at[i,'Opinion']-self.Oi)
            if V == Comp:
                vota = random.randint (i, vota)
            elif tipo == "Positivo":
                if V < Comp:
                    vota = i
                    Comp = V
            elif tipo == "Negativo":
                if V > Comp:
                    vota = i
                    Comp = V                
        return vota # Me devuelve la id de la idea a votar 

        def votar (self, ideas):
            self.IdeasVistas.append (ideas)
            ideas2 = ideas.copy ()
            vota = []
            votos = self.ListaVotos(VotosPos, VotosNeg)
            for i in votos:
                if i == 1:
                    va = self.algvotar (ideas2, "Positivo")
                    ideas2.drop (va)
                    vota.append (va)
                if i == -1:
                    va = self.algvotar (ideas2, "Negativo")
                    ideas2.drop (va)
                    vota.append (va)
            self.IdeasVotadas.append (vota)
            self.Valencias.append (votos)
            devuelve = pd.DataFrame(votos, vota)
            return devuelve

        
        
## >>>>>>>>>>>>>>>>>>>>>>>>>><
# np.random.choice([1,2,3,4,5], 3, p=[0.01, 0, 0.99, 0, 0])
# print (len(DataSave.Opinion[3]))

### Metodos se seleccion de ideas

Esta funcion define la manera en que se va a controlar las ideas que le van a ser presentadas a un agente. Existen dos metodos con que se pueden seleccionar las ideas dependiendo del argumento `tipo` pasado a la funcion.

* Metodo uno (`tipo = 'menos vistas'`): Comprende elegir aquellas ideas que tengan menor cantidad de vistas
* Metodo dos (`tipo = 'ratio'`): Comprende elegir en partes iguales, la mitad de las ideas segun la cantidad de vistas igual que con el metodo uno y la otra mitad de las ideas segun el ratio de votos positivos y cantidad de vistas.   

In [6]:
def metodo_de_seleccion (lista, tipo):
    
    if tipo == "menos vistas":
        lista.sort_values(by='Vistas', inplace=True)
        Devuelve = lista.iloc[:QOpinionesExpuestas, :5]
        return Devuelve

    if tipo == "ratio":
        lista.sort_values(by=['Vistas'], ascending = True, inplace = True)
        OpinionesMasVotadas = lista.iloc[:10, :5] # 
        OpinionesMasVotadasSeleccionadas = OpinionesMasVotadas.sample (n= math.floor (QOpinionesExpuestas / 2))
        RestoDeOpiniones = lista.iloc[10:, :]
        RestoDeOpiniones.sort_values(by='Ratio', ascending=False, inplace=True)
        OpinionesConMayorRatio = RestoDeOpiniones.iloc[:10, :5]
        OpinionesConMayorRatioSeleccionadas = OpinionesConMayorRatio.sample (n= math.ceil (QOpinionesExpuestas / 2))
        OpinionesParaVotar = pd.concat([OpinionesMasVotadasSeleccionadas, OpinionesConMayorRatioSeleccionadas])
        return OpinionesParaVotar
    
        

### Simulacion Nula

Se crea un diccionario para albergar a todos los agentes que van a componer la muestra. Para crear el diccionario se itera sobre el rango del tamaño de la muestra a usar (estando los distintos tamaños definidos en una lista) y se genera un nuevo elemento del diccionario (una instancia de la clase correspondiente a algun tipo de agente) con un indice igual al ID del sujeto. 

#### Variables de la muestra

Para la muestra de la simulacion es necesario definir algunas variables que van a determinar el tipo de distribucion que tendra la opinion de los agentes. Estas variables son:

* `Tipo` = Representa la distirbucion que va a tomar la opinion del sujeto,  puede ser  'uniforme', ' univariada' o 'multivariada'.
* `Media` = Debe ser una lista, y debe tener [start,stop] para el caso uniforme, y 1, 2 o 3 numeros para las univariadas y las multivariadas  

In [7]:
Tipo = 'uniforme' # Puede ser  'uniforme', ' univariada' o 'multivariada'
Media = [-1,1]  # Debe ser una lista, y debe tener [start,stop] para el caso uniforme, y 1, 2 o 3 numeros para las univariadas y las multivariadas

#### Creo la muestra

In [8]:
Muestra_Completa = {}
for k in range (1,TamañoDeLaMuestra[1]):
    Muestra_Completa [k*111] = AgenteNulo (Tipo, Media, str(uuid4()))

#### Variables de entorno

Para el contexto de la simulacion es necesario definir algunas variables que van a determinar el contexto de la plataforma en la cual los agentes van a interactuar. Estas variables son:

* `QOpinionesExpuestas` = La cantidad de opiniones que va a ver cada agente. 
* `QVecesMuestroOpinion` = La cantidad minima de veces que quiero que se muestre alguna opinion, toda opinion de esta manera tiene que mostrarse al menos esta cantidad de veces. 
* `VotosPos` = Cuantos votos positivos tiene disponible cada agente.
* `VotosNeg` = Cuantos votos negativos tiene disponible cada agente.
* `QIdeasAVotar` = La cantidad total de ideas que un agente va a votar lo cual esta dado por la suma de la cantidad de votos negativos y positivos disponible para cada agente. 

In [9]:
QOpinionesExpuestas = 5 
QVecesMuestroOpinion = 15
VotosPos = 2
VotosNeg = 1
QIdeasAVotar = VotosPos + VotosNeg

#### Simulacion

Para la simulacion es necesario definir algunas variables. Estas variables son:

* `Muestra_Simulacion` = Es una copia de la muestra completa que se utilizara para esta simulacion en especifico. 
* `DataSave` = Es un array que almacenara la ID del sujeto, su opinion, la cantidad de votos (positivos y negativos), la cantidad de vistas, y el ratio (Vistas/Votos).
* `MinutoAMinuto` = Es un array que almacena lo que va sucediendo paso por paso en la simulacion.
* `OpinanNoVotan` y `TiempoSimulacion` son dos variables que sirven a modo funcional de la simulacion.

In [10]:
Muestra_Simulacion = Muestra_Completa.copy() # Creo una copia de la muestra original, por si se crean varias simulaciones diferentes
DataSave = pd.DataFrame(columns=['ID', 'TipoDeDistribucion', 'Media/s', 'DesvioEstandar', 'Opinion', 'VotosPos', 'VotosNeg', 'Vistas', 'Ratio'])
MinutoAMinuto = pd.DataFrame(columns=['Sujeto', 'TipoDeDistribucion', 'Media/s', 'DesvioEstandar', 'Opinion Sujeto', 'ID Idea Mostrada', 'Idea Mostrada', 'Otras opiniones que vio', 'Voto', 'Distancia'])
OpinanNoVotan = 0
TiempoSimulacion = 0


for i in Muestra_Simulacion:
    ParticipanteActual = i
    
    if OpinanNoVotan < QOpinionesExpuestas: # Si hay menos opiniones que las necesarias para mostrar los individuos van a opinar, no votar.
        TupleOi = Muestra_Simulacion.get(i).get_Oi()
        DataSave.at [OpinanNoVotan] = Muestra_Simulacion.get(i).get_ID(), TupleOi [0], TupleOi [1], TupleOi [2], TupleOi [3], 0, 0, 0, 0
        OpinanNoVotan +=1
    
    else:
        TupleOi = Muestra_Simulacion.get(i).get_Oi()
        OpinionesFiltradas = DataSave[DataSave.Vistas < QVecesMuestroOpinion] # Filtro las ideas que tengan menos vistas del limite asignado
        OpinionesParaVotar =  metodo_de_seleccion (OpinionesFiltradas, "menos vistas") # Creo un dataframe con las ideas que va a ver el sujeto
        DataSave.at [i] = Muestra_Simulacion.get(i).get_ID(), TupleOi [0], TupleOi [1], TupleOi [2], TupleOi [3], 0, 0, 0, 0
        VotoActual = Muestra_Simulacion.get(i).votar(OpinionesParaVotar)
        
        for i in VotoActual.index:
            
            if VotoActual.loc[i].at[0] == 1:
                DataSave.at [i,'VotosPos'] = DataSave.at [i,'VotosPos'] + 1 # Le sumo un voto a la idea.
            
            if VotoActual.loc[i].at[0] == -1:
                DataSave.at [i,'VotosNeg'] = DataSave.at [i,'VotosNeg'] + 1 # Le sumo un voto a la idea. 
        
        for i in OpinionesParaVotar.index:
            OpinionI = i
            DataSave.at [i,'Vistas'] = DataSave.at [i,'Vistas'] + 1 # Le sumo una vista a la ideas expuestas
            
            ################################################################ DEBERIA CONFIGURAR ALGO PARA VER COMO AFECTAN LOS VOTOS NEGATIVOS EN EL RATIO?
            DataSave.at [i, 'Ratio'] = DataSave.at[i, 'VotosPos'] / DataSave.at [i, 'Vistas']
            
            # Armo un dataframe con el minuto a minuto de lo que fue pasando en la simulacion
            CopiaOpiniones = OpinionesParaVotar.index.tolist()
            CopiaOpiniones.remove(i)
            #MinutoAMinuto.at [TiempoSimulacion] = Muestra_Simulacion.get(ParticipanteActual).get_ID(), Muestra_Simulacion.get(ParticipanteActual).get_Oi(), i, OpinionesParaVotar.at [i,'Opinion'], CopiaOpiniones, 0, Muestra_Simulacion.get(ParticipanteActual).comparar(OpinionesParaVotar.at [i,'Opinion']) 
            if i in VotoActual.index:
                for i in VotoActual.index:
                    if i == OpinionI:
                        MinutoAMinuto.at [TiempoSimulacion,'Voto'] = VotoActual.at [i,0]  
            TiempoSimulacion +=1

# print (DataSave.Opinion)
print (DataSave)
# ##### Visualizo la data
# print ("-------------------------------------------------------------------------------------")
# #print (DataSave.sort_values(by=['Votos'], ascending=False))
# DataSave.plot(kind='scatter',x='Opinion',y='VotosPos',color='green')
# DataSave.plot(kind='scatter',x='Opinion',y='VotosNeg',color='red')
# DataSave.plot(kind='scatter',x='Opinion',y='Vistas',color='blue')
# plt.show()

C:\Users\Mariano\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:993: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.array(value)


                                         ID TipoDeDistribucion  Media/s  \
0      1e557196-dc50-4ffd-9933-b45717fdd336           uniforme  [-1, 1]   
1      b928be33-0678-4ae0-a4ba-07fd94fccb74           uniforme  [-1, 1]   
2      855aa575-426d-4597-9e5c-a2529b3ce4b4           uniforme  [-1, 1]   
3      e92bf383-386d-4380-866b-cd79278dc530           uniforme  [-1, 1]   
4      269e4851-93ae-4d1e-b6e1-948cda5f5ffd           uniforme  [-1, 1]   
...                                     ...                ...      ...   
21645  df5452f4-ff46-4866-bad1-ffc9ec561274           uniforme  [-1, 1]   
21756  e41ade38-155e-47a0-b41a-e6d7cfeb8168           uniforme  [-1, 1]   
21867  78446f2d-44eb-46d7-ab65-27209dee8a32           uniforme  [-1, 1]   
21978  85658df9-0799-4a38-9099-e7323f497e4a           uniforme  [-1, 1]   
22089  9ffd7a8c-88b3-4d11-ae00-1caaf02c68c7           uniforme  [-1, 1]   

      DesvioEstandar                 Opinion VotosPos VotosNeg Vistas  \
0                0.5   [-0

In [11]:
# class Agente (object):
#     def __init__ (self, tipo, media, ID, de = 0.5):
#         self.Oi = self.set_Oi (tipo, media, de)
#         self.ID = ID
#         self.IdeasVistas = []
#         self.IdeasVotadas = []
#         self.Valencias = []

# ###############################################################################################

# # Le paso el argumento tipo, que puede ser uniforme, univariada o multivariada, y el argumento media, que tiene que ser una lista.
# # Si es uniforme devuelve un numero tomando como start el primer numero de la lista y como end el segundo
# # Si es univariada se fija si la lista de medias tiene una sola, devuelve un numero con una normal con esa media
# # si tiene mas numeros devuelve un numero para cada una de las medias
# # si es multivariada se fija si tiene un solo numero, devuelve tres numeros con esa media (diagonal del array)
# # si tiene varios numeros hace un random choice 3 veces de ese numero de medias.  

#     def set_Oi (self, tipo, media, de):
        
#         if tipo != "uniforme" and tipo != "univariada" and tipo != "multivariada":
#             raise Exception("El tipo de opinion solo puede ser ""uniforme"", ""univariada"" o ""multivariada")
        
#         elif isinstance(media, list) == False:
#             raise Exception("El argumento media solo acepta listas")
        
#         opinion = pd.DataFrame()
#         f = 0
#         c = 0
        
#         if tipo == "uniforme":
#             if len(media) != 2:
#                 raise Exception("La media del tipo uniforme debe contener un start y un end")
#             opinion.at[f,c] = (random.uniform (media[0],media[1]))
            
#         if tipo == "univariada":
#             if len(media) == 1:
#                 opinion.at[f,c] = (np.random.normal (media))
                
#             else:
#                 for i in media:
#                     opinion.at[f,c] = (np.random.normal (i))
#                     f += 1
                    
#         if tipo == "multivariada":
#             if len(media) == 1:
#                 for i in range (3):
#                     opinion.at[f,c] = (np.random.normal (media))
#                     f += 1
#                     c += 1
                    
#             else:
#                 for i in range (3):
#                     opinion.at[f,c] = (np.random.normal (random.choice (media)))
#                     f += 1
#                     c += 1
                
#         ListaOpiniones = opinion.values.tolist()
        
#         return ListaOpiniones #devuelve una lista con las opiniones

# ###############################################################################################

#     def append_IdeasVistas (self, IdeaVista):
#         self.IdeasVistas.append (IdeaVista)

#     def append_IdeasVotadas (self, IdeaVotada):
#         self.IdeasVotadas.append (IdeaVotada)   

#     def append_Valencias (self, Valencias):
#         self.Valencias.append (Valencias) 

#     def ListaVotos (self, VotosPos, VotosNeg):
#         votos = VotosPos * [1] + VotosNeg * [-1]
#         return votos

#     def comparar (self, idea): # Devuelve la distancia entre la opinion que se le pase y la del sujeto
#         Distancia = abs(idea-self.Oi)
#         return Distancia
    
#     def get_Oi (self):
#         return self.Oi
    
#     def get_IdeasVistas (self):
#         return self.IdeasVistas

#     def get_ID (self):
#         return self.ID
    
#     def clean_IdeasVistas (self):
#         self.IdeasVistas = []